## Affine / Softmaxレイヤーの実装

In [5]:
import numpy as np

### Affine レイヤー

ニューラルネットワークの順伝播では、

$$Y = X W + B$$

を用いた。このような変換をアフィン変換といい、その処理を行うレイヤーを Affineレイヤーと呼ぶ。

In [ ]:
# バッチ版Affineレイヤー
class Affine:

    def __init__(self, W: np.ndarray, b: np.ndarray):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x: np.ndarray):
        self.x = x
        return np.dot(x, self.W) + self.b

    def backward(self, dout: np.ndarray):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        # dBはY1からYnまでの偏微分の和
        self.db = np.sum(dout, axis=0)
        return dx

### Softmax-with-Loss レイヤー

In [ ]:
def cross_entropy_error(y, t):
    """予測値yと正解ラベルtの交差エントロピー誤差を返す。１次元のベクトルに対しても、行列（ミニバッチ）に対しても適用可。"""

    # バッチがない場合（yが１次元のベクトルの場合）も、1×n 行列に変換しておく
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換する
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax(x):
    """ミニバッチに対しても適用できるsoftmax関数"""
    x = x - np.max(x, axis=-1, keepdims=True)  # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

In [ ]:
# バッチ対応で実装している
class SoftmaxWithLoss:

    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x: np.ndarray, t: np.ndarray):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        # バッチサイズで割り、データ１個あたりの誤差が伝播するようにしている
        dx = (self.y - self.t) / batch_size
        return dx